In [1]:
# unset PYTHONPATH first
from ko_text import *
from ko_crawler import *

In [2]:
nlp = NLP()

### Morphs 

In [3]:
train_df = pd.read_csv('Data/morphs/train_df_morphs.csv',
                       usecols = ['Token','Section'],
                       dtype = {'Section' : 'category'},
                       converters = {'Token' : ast.literal_eval},
                     )

test_df = pd.read_csv('Data/morphs/test_df_morphs.csv',
                       usecols = ['Token','Section'],
                       dtype = {'Section' : 'category'},
                       converters = {'Token' : ast.literal_eval},
                     )

### Nouns 

In [3]:
train_df = pd.read_csv('Data/nouns/train_df_nouns.csv',
                       usecols = ['Token','Section'],
                       dtype = {'Section' : 'category'},
                       converters = {'Token' : ast.literal_eval},
                     )

test_df = pd.read_csv('Data/nouns/test_df_nouns.csv',
                       usecols = ['Token','Section'],
                       dtype = {'Section' : 'category'},
                       converters = {'Token' : ast.literal_eval},
                     )

## **제목만 있고 특수 문자를 제거하면 기사 내용이 전혀 없는 뉴스도 존재**

#### 내용이 없는 뉴스들은 제거한다

In [4]:
none_idx_ls = []
for idx, token in enumerate(train_df['Token']):
    if not token:
        none_idx_ls.append(idx)        

In [5]:
len(none_idx_ls)

2124

In [6]:
train_df.drop(none_idx_ls, inplace = True)

#### **Test_df 도 동일하게 제거해줌**

In [7]:
none_idx_ls = []

for idx, token in enumerate(test_df['Token']):
    if not token:
        none_idx_ls.append(idx)

In [10]:
len(none_idx_ls)

396

In [ ]:
test_df.drop(none_idx_ls, inplace= True)

# 뉴스 기사가 짧은 경우도 존재

In [12]:
train_df['Num of Tokens'] = [len(token) for token in train_df['Token']]
test_df['Num of Tokens'] = [len(token) for token in test_df['Token']]

#### 뉴스 기사에 토큰이 하나 밖에 없는 경우도 존재함

In [13]:
train_df['Num of Tokens'].describe()

count    87876.000000
mean       209.709921
std        157.452864
min          1.000000
25%        112.000000
50%        176.000000
75%        268.000000
max       4327.000000
Name: Num of Tokens, dtype: float64

In [14]:
test_df['Num of Tokens'].describe()

count    22103.000000
mean       207.830430
std        155.140838
min          1.000000
25%        113.000000
50%        177.000000
75%        263.000000
max       4140.000000
Name: Num of Tokens, dtype: float64

#### 토큰의 수가 50개 이하인 기사는 전체의 5.5% 수준

In [37]:
np.sum([train_df['Num of Tokens'] <= 50]) / len(train_df)

0.055908325367563384

#### 토큰이 50개인 기사의 길이는 대략 2줄 정도

In [27]:
' '.join(train_df[train_df['Num of Tokens'] == 50].iloc[100].tolist()[0])

'농협 은행 추석 연휴 맞아 현금 유가 증권 귀중 이용 수수료 없이 보관 해주는 서비스 운영 한다고 밝혔다 서비스 오는 전국 영업 에서 실시 한다 가까운 영업 금고 남아있는지와 보호 예수 절차 문의 하고 이용 하면 된다 자세한 내용 농협 인터넷 뱅킹 홈페이지 영업 고객 행복 센터 문의 하면 된다'

#### 토큰의 수가 100개 미만인 기사는 전체의 20% 수준

In [32]:
np.sum([train_df['Num of Tokens'] < 50]) / len(train_df)

0.053473075697573855

#### 토큰이 50개인 기사의 길이는 대략 2줄 정도

In [35]:
' '.join(train_df[train_df['Num of Tokens'] == 50].iloc[102].tolist()[0])

'┬┬┬ 매매 기준 매매 기준 미국 달러 뉴질랜드 달러 일본 홍콩 달러 유럽 통화 단위 유로 스웨덴 크로네 덴마크 크로네 노르웨이 크로네 사우디아라비아 리알 쿠웨이트 디나르 바레인 디나르 아랍에미리트 디르함 태국 바트 싱가포르 달러 영국 파운드 인도 루피 스위스 프랑 말레이시아 링깃 캐나다 달러 인도네시아 루피아 호주 달러 중국 위안'

## 토큰의 수가 50개 이하인 기사는 표본에서 제거

In [38]:
train_df = train_df[train_df['Num of Tokens'] > 50]
test_df = test_df[test_df['Num of Tokens'] > 50]

### 각 섹션 별로 남은 기사의 비율 체크

체크한 결과 경제 기사가 문제가 많았던 것으로 확인 됨,

약 25%의 기사에 문제가 있었음..

In [39]:
Counter(train_df['Section'])

Counter({'IT과학': 9836,
         '경제': 8050,
         '국제': 9677,
         '기업': 9604,
         '문화': 9313,
         '부동산': 9714,
         '사회': 9325,
         '정치': 8875,
         '증권': 8569})

In [41]:
Counter(test_df['Section'])

Counter({'IT과학': 2471,
         '경제': 1873,
         '국제': 2422,
         '기업': 2405,
         '문화': 2310,
         '부동산': 2451,
         '사회': 2334,
         '정치': 2307,
         '증권': 2220})

### **저장 공간 절약을 위해, ['단어', '단어'] 꼴로 저장된 토큰을 '단어 단어' 꼴로 바꿔준다**

In [42]:
train_df['Token'] = [' '.join(doc) for doc in train_df['Token']]
test_df['Token'] = [' '.join(doc) for doc in test_df['Token']]

In [43]:
train_df['Token'][0]

'농협 금융 지주 김광수 회장 서대문 본사 에서 농협 금융 자회사 대표 종합 경영 성과 분석 회의 했다고 밝혔다 회의 에서는 농협카드 농협 캐피탈 체질개선 질적 성장 방안 주로 논의 됐다 회장 농협카드 조직 효율 성과 인적 전문성 높이 독립 성과 평가 통해 책임 경영 체제 확립 라고 주문 했다 상품 고객 수익 분석 고도화 일반 카드 수준 경영 인프라 축하 라고 당부 했다 회장 농협 캐피탈 관련 해서는 산업 금융 건설 경기 악화 염두 두고 자산 건전성 개선 노력 어가 기업 금융 취급 부터 위험관리 철저히 해야 한다고 강조 했다 회장 연말 인사 에서는 업무 경력 직무 전문성 우선 고려 하겠다고 했다'

# morph

In [44]:
train_df.to_csv('Data/train_morphs_final.csv', index = False)
test_df.to_csv('Data/test_morphs_final.csv', index = False)

# noun

In [23]:
train_df.to_csv('Data/Train_nouns_final.csv', index = False)
test_df.to_csv('Data/Test_nouns_final.csv', index = False)